# Require

- pytraj: >= 1.0.9
- nglview master branch: https://github.com/arose/nglview
- jamber master branch: https://github.com/hainm/jamber
- AmberTools

# Import stuff

In [10]:
from jamber import leap, sander

# use "ambertools" instead of "jamber"?

# Leap build

In [11]:
leap.build("""
source leaprc.protein.ff14SBonlysc
seq = sequence{ ALA ALA ALA}
saveamberparm seq prmtop seq.rst7
""")

# Visualize leap outputs

In [12]:
import pytraj as pt

traj = pt.load('seq.rst7', top='prmtop')
traj.visualize()

# Create md input

In [13]:
%%file md.in

Langevin dynamics simulations
&cntrl
   ntx = 1, irest=0,
   imin = 0, nstlim = 500000, dt = 0.002,
   ntt = 3, gamma_ln=1., temp0 = 300.0, tempi=300.,
   ntc= 2, ntf = 2,
   igb=8, ntb = 0,
   ntwx = 500, ntwe = 0, ntwr = 500, ntpr = 500,
   cut = 999.0,
   nscm = 500,
/
&end

Overwriting md.in


# Run sander

In [14]:
job = sander.run("-O -p {} -c {} -i md.in".format('prmtop', 'seq.rst7'))

# Real time monitoring simulation

In [15]:
from nglview.sandbox.amber import AmberMD

ambermd = AmberMD(top='prmtop', restart='restrt', reference='seq.rst7')
ambermd.initialize()

In [16]:
ambermd.update()

# Stop sander job

In [17]:
# un-comment
job.terminate()